<a href="https://colab.research.google.com/github/TechKnight10/Summarization-model-using-Hugging-face-xlsum-and-ai4bharat-IndicBART/blob/main/IndicBARTsummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART-XLSum", do_lower_case=False, use_fast=False, keep_accents=True)
model = MBartForConditionalGeneration.from_pretrained("ai4bharat/IndicBART-XLSum")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [ ]:
file_path1 = 'urls_and_articles_summary_hindi.csv'
try:
    df1 = pd.read_csv(file_path1, encoding='utf-8-sig')
    print(df1.head())
except FileNotFoundError:
    print(f"The file {file_path1} was not found.")
except UnicodeDecodeError:
    print("Unicode decode error: Try changing encoding to 'utf-8'.")

# Additional check for the contents of 'df1' for debugging
print("Data loaded successfully. Columns:", df1.columns)
print("First few rows of data:", df1.head())

                                          URL Source  \
0  https://www.bbc.com/hindi/sport/story/2008/03/...   
1           https://www.bbc.com/hindi/india-45015573   
2  https://www.bbc.com/hindi/news/story/2008/11/0...   
3  https://www.bbc.com/hindi/india/2015/02/150208...   
4  https://www.bbc.com/hindi/sport/story/2008/08/...   

                                             Article  \
0  दोनों ने भारत के वीनू मनकड और पंकज राय का 52 स...   
1  ये खबर इंडियन एक्सप्रेस के पहले पन्ने पर है. प...   
2  माना जा रहा है कि हाल में अमरीकी राष्ट्रपति चु...   
3  लेकिन जब हम अपने डेस्क पर बैठकर ख़बर बनाते हैं...   
4  ओलंपिक खेल की संयोजक समिति के अधिकारियों ने मा...   

                                             Summary  
0  दक्षिण अफ्रीका के कप्तान ग्रेम स्मिथ और नील मै...  
1  भारतीय जनता पार्टी ने कहा है कि असम के बाद वो ...  
2  अमरीका में कई दिनों की मंदी के बाद शेयर बाज़ार...  
3  हम रोज़ाना गालियां सुनते हैं. देते हैं. गुस्से...  
4  बीजिंग ओलंपिक में अभिनव बिंद्रा के स्वर्ण पदक ..

In [ ]:
def summarize_article(article):
    try:
        # Tokenize input article and ensure it's on the correct device (GPU or CPU)
        inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
        print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

        # Check and adjust for max length
        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]


        inp = inp.to(device)

        # Check tensor shape
        print(f"Input tensor shape: {inp.shape}")

        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]

        model.eval()
        model.to(device)
        model_output= model.generate(inp, use_cache=True, num_beams=4, max_length=60, min_length=20, length_penalty=1.0, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2hi>"))

        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"

df1['Summary'] = df1['Article'].apply(summarize_article)

print(df1[['Article', 'Summary']].head())
df1.to_csv("summarized_articles_hindi.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 308]), on device: cpu
Input tensor shape: torch.Size([1, 308])
Input tensor shape: torch.Size([1, 756]), on device: cpu
Input tensor shape: torch.Size([1, 756])
Input tensor shape: torch.Size([1, 345]), on device: cpu
Input tensor shape: torch.Size([1, 345])
Input tensor shape: torch.Size([1, 798]), on device: cpu
Input tensor shape: torch.Size([1, 798])
Input tensor shape: torch.Size([1, 432]), on device: cpu
Input tensor shape: torch.Size([1, 432])
Input tensor shape: torch.Size([1, 333]), on device: cpu
Input tensor shape: torch.Size([1, 333])
Input tensor shape: torch.Size([1, 364]), on device: cpu
Input tensor shape: torch.Size([1, 364])
Input tensor shape: torch.Size([1, 758]), on device: cpu
Input tensor shape: torch.Size([1, 758])
Input tensor shape: torch.Size([1, 333]), on device: cpu
Input tensor shape: torch.Size([1, 333])
Input tensor shape: torch.Size([1, 367]), on device: cpu
Input tensor shape: torch.Size([1, 367])
Input tensor shape: 

In [ ]:
file_path_2 = 'urls_and_articles_summary_bengali.csv'
try:
    df2 = pd.read_csv(file_path_2, encoding='utf-8-sig')
    print(df2.head())
except FileNotFoundError:
    print(f"The file {file_path_2} was not found.")
except UnicodeDecodeError:
    print("Unicode decode error: Try changing encoding to 'utf-8'.")

# Additional check for the contents of 'df' for debugging
print("Data loaded successfully. Columns:", df2.columns)
print("First few rows of data:", df2.head())

                                  URL Source  \
0  https://www.bbc.com/bengali/news-56713767   
1  https://www.bbc.com/bengali/news-56533637   
2  https://www.bbc.com/bengali/news-56631353   
3  https://www.bbc.com/bengali/news-47189627   
4  https://www.bbc.com/bengali/news-43949363   

                                             Article  \
0  বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসল...   
1  বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুল...   
2  ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জ...   
3  ২০১৮ সালের নির্বাচন প্রত্যাখ্যান করেছে বিএনপি।...   
4  কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের ...   

                                             Summary  
0  ইসলামী শিক্ষাবিদ এবং যুক্তরাজ্যের জাতীয় স্বাস...  
1  বাংলাদেশে ভারতীয় প্রধানমন্ত্রী নরেন্দ্র মোদীর...  
2  বাংলাদেশে রাজধানী ঢাকার নিকটবর্তী নারায়ণগঞ্জে...  
3  বাংলাদেশে গত ৩০শে ডিসেম্বর অনুষ্ঠিত সংসদ নির্ব...  
4  বিয়ে টিকিয়ে রাখার ক্ষেত্রে স্বামী ও স্ত্রীর ...  
Data loaded successfully. Columns: Index(['U

In [ ]:
def summarize_article(article):
    try:
        # Tokenize input article and ensure it's on the correct device (GPU or CPU)
        inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
        print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

        # Check and adjust for max length
        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]


        inp = inp.to(device)

        # Check tensor shape
        print(f"Input tensor shape: {inp.shape}")

        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]

        model.eval()
        model.to(device)
        model_output= model.generate(inp, use_cache=True, num_beams=4, max_length=60, min_length=20, length_penalty=1.0, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2bn>"))

        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"

df2['Summary'] = df2['Article'].apply(summarize_article)

print(df2[['Article', 'Summary']].head())
df2.to_csv("summarized_articles_bengali.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 869]), on device: cpu
Input tensor shape: torch.Size([1, 869])
Input tensor shape: torch.Size([1, 424]), on device: cpu
Input tensor shape: torch.Size([1, 424])
Input tensor shape: torch.Size([1, 249]), on device: cpu
Input tensor shape: torch.Size([1, 249])
Input tensor shape: torch.Size([1, 749]), on device: cpu
Input tensor shape: torch.Size([1, 749])
Input tensor shape: torch.Size([1, 375]), on device: cpu
Input tensor shape: torch.Size([1, 375])
Input tensor shape: torch.Size([1, 510]), on device: cpu
Input tensor shape: torch.Size([1, 510])
Input tensor shape: torch.Size([1, 314]), on device: cpu
Input tensor shape: torch.Size([1, 314])
Input tensor shape: torch.Size([1, 237]), on device: cpu
Input tensor shape: torch.Size([1, 237])
Input tensor shape: torch.Size([1, 464]), on device: cpu
Input tensor shape: torch.Size([1, 464])
Input tensor shape: torch.Size([1, 168]), on device: cpu
Input tensor shape: torch.Size([1, 168])
Input tensor shape: 

In [ ]:
file_path_3 = 'urls_and_articles_summary_gujarati.csv'
try:
    df3 = pd.read_csv(file_path_3, encoding='utf-8-sig')
    print(df3.head())
except FileNotFoundError:
    print(f"The file {file_path_3} was not found.")
except UnicodeDecodeError:
    print("Unicode decode error: Try changing encoding to 'utf-8'.")

# Additional check for the contents of 'df' for debugging
print("Data loaded successfully. Columns:", df3.columns)
print("First few rows of data:", df3.head())

                                          URL Source  \
0        https://www.bbc.com/gujarati/india-44436218   
1        https://www.bbc.com/gujarati/india-56384394   
2        https://www.bbc.com/gujarati/india-46011192   
3  https://www.bbc.com/gujarati/international-485...   
4        https://www.bbc.com/gujarati/india-52558025   

                                             Article  \
0  જોકે, રાજપૂતાના રાઇફલ્સની બીજી બટાલિયનમાં જોડા...   
1  સામાચાર એજન્સી એએનઆઈ પ્રમાણે આગ લાગવાની આ ઘટના...   
2  શશી થરૂરની ટિપ્પણીથી સોશિયલ મીડિયામાં ચર્ચા થર...   
3  જ્યારે વિરાટ કોહલીની મજબૂત ટીમ તેના જેવી જ મજબ...   
4  સ્વાતિ માલીવાલ ફોટો શૅરિંગ સોશિયલ નેટવર્કિંગ સ...   

                                             Summary  
0  ભારતીય આર્મીની જે બટાલિયનમાં પિતા લાન્સ નાયક હ...  
1  દિલ્હી દહેરાદૂન એક્સપ્રેસના કોચ નંબર C4 માં આગ...  
2  કોંગ્રેસના નેતા શશી થરૂરની વડા પ્રધાન પરની ટિપ...  
3  વર્લ્ડ કપ જીતવાના ભારતના અભિયાનની આકરી કસોટી ર...  
4  ભારતમાં સોશિયલ મીડિયામાં છેલ્લા બે દિવસથી #Boy..

In [ ]:
def summarize_article(article):
    try:
        # Tokenize input article and ensure it's on the correct device (GPU or CPU)
        inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
        print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

        # Check and adjust for max length
        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]


        inp = inp.to(device)

        # Check tensor shape
        print(f"Input tensor shape: {inp.shape}")

        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]

        model.eval()
        model.to(device)
        model_output= model.generate(inp, use_cache=True, num_beams=4, max_length=60, min_length=20, length_penalty=1.0, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2gu>"))

        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"

df3['Summary'] = df3['Article'].apply(summarize_article)

print(df3[['Article', 'Summary']].head())
df3.to_csv("summarized_articles_gujarati.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 1485]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 1066]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 554]), on device: cpu
Input tensor shape: torch.Size([1, 554])
Input tensor shape: torch.Size([1, 1095]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 1494]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 1140]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 446]), on device: cpu
Input tensor shape: torch.Size([1, 446])
Input tensor shape: torch.Size([1, 544]), on device: cpu
Input tensor shape: torch.Size([1, 544])
Input tensor shape: torch.Size([1, 905]), on device: cpu
Input tensor shape: torch.Size([1, 905])
Input tensor shape: torch.Size([1, 105]), on device: cpu
Input tensor shape: torch.Size([1, 105])
Input tens

In [ ]:
file_path_4 = 'urls_and_articles_summary_marathi.csv'
try:
    df4 = pd.read_csv(file_path_4, encoding='utf-8-sig')
    print(df4.head())
except FileNotFoundError:
    print(f"The file {file_path_4} was not found.")
except UnicodeDecodeError:
    print("Unicode decode error: Try changing encoding to 'utf-8'.")

# Additional check for the contents of 'df' for debugging
print("Data loaded successfully. Columns:", df4.columns)
print("First few rows of data:", df4.head())

                                   URL Source  \
0  https://www.bbc.com/marathi/media-47201457   
1  https://www.bbc.com/marathi/india-55121329   
2  https://www.bbc.com/marathi/india-56420758   
3  https://www.bbc.com/marathi/india-41641154   
4  https://www.bbc.com/marathi/india-50862832   

                                             Article  \
0  लवकर केस पांढरे होत असतील तर त्याला वैद्यकीय भ...   
1  स्टीव्हन स्मिथने वेगवान शतक झळकावलं. सिडनी क्र...   
2  कोरोना लसीकरण खुद्द पंतप्रधान मोदींनीही 1 मार्...   
3  परदेशातील नातेवाईकांना पाठविला जातो दिवाळीचा फ...   
4  संगीतकार अजय अतुल यांचा भारतातल्या 100 सेलिब्र...   

                                             Summary  
0  कमी वयात केस पांढरे होणं सामान्य बाब होत चालली...  
1  तडाखेबंद बॅटिंगच्या बळावर ऑस्ट्रेलियाने दुसऱ्य...  
2  भारतात कोव्हिड-19चं लसीकरण सुरू होऊन आता दोन म...  
3  आईच्या हातचा फराळ ही जगात भारी गोष्ट असते. नरक...  
4  फोर्ब्स इंडियाने प्रसिद्ध केलेल्या 'Celebrity ...  
Data loaded successfully. Columns: Ind

In [ ]:
def summarize_article(article):
    try:
        # Tokenize input article and ensure it's on the correct device (GPU or CPU)
        inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
        print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

        # Check and adjust for max length
        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]


        inp = inp.to(device)

        # Check tensor shape
        print(f"Input tensor shape: {inp.shape}")

        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]

        model.eval()
        model.to(device)
        model_output= model.generate(inp, use_cache=True, num_beams=4, max_length=60, min_length=20, length_penalty=1.0, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2mr>"))

        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"

df4['Summary'] = df4['Article'].apply(summarize_article)

print(df4[['Article', 'Summary']].head())
df4.to_csv("summarized_articles_Marathi.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 181]), on device: cpu
Input tensor shape: torch.Size([1, 181])
Input tensor shape: torch.Size([1, 887]), on device: cpu
Input tensor shape: torch.Size([1, 887])
Input tensor shape: torch.Size([1, 1247]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 912]), on device: cpu
Input tensor shape: torch.Size([1, 912])
Input tensor shape: torch.Size([1, 482]), on device: cpu
Input tensor shape: torch.Size([1, 482])
Input tensor shape: torch.Size([1, 152]), on device: cpu
Input tensor shape: torch.Size([1, 152])
Input tensor shape: torch.Size([1, 849]), on device: cpu
Input tensor shape: torch.Size([1, 849])
Input tensor shape: torch.Size([1, 1129]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 956]), on device: cpu
Input tensor shape: torch.Size([1, 956])
Input tensor shape: torch.Size([1, 880]), on device: cpu
Input tensor shape: torch.Size([1, 880])
Input tensor sha

In [ ]:
file_path_5= 'urls_and_articles_summary_tamil.csv'
try:
    df5 = pd.read_csv(file_path_5, encoding='utf-8-sig')
    print(df5.head())
except FileNotFoundError:
    print(f"The file {file_path_5} was not found.")
except UnicodeDecodeError:
    print("Unicode decode error: Try changing encoding to 'utf-8'.")

# Additional check for the contents of 'df' for debugging
print("Data loaded successfully. Columns:", df5.columns)
print("First few rows of data:", df5.head())

                                     URL Source  \
0  https://www.bbc.com/tamil/sri-lanka-50684925   
1     https://www.bbc.com/tamil/global-51812206   
2      https://www.bbc.com/tamil/india-43637453   
3    https://www.bbc.com/tamil/science-52339766   
4      https://www.bbc.com/tamil/india-47938063   

                                             Article  \
0  தீர்ப்பு வெளியான பிறகு செய்தியாளர்களிடம் பேசிய...   
1  சித்தரிப்புக்காக வெனிசுவேலா நாட்டில் ஏறத்தாழ 5...   
2  எங்களின் போராட்டம் மக்களின் எதார்த்த வாழ்க்கைய...   
3  சரி. கொரோனா வைரஸ் தொற்றால் பாதிக்கப்பட்ட ஒருவர...   
4  தீவிர பிரச்சாரத்திற்கு மத்தியில் பிபிசி தமிழின...   

                                             Summary  
0  உள்ளாட்சித் தேர்தல் தொடர்பாக உச்ச நீதிமன்றம் வ...  
1  எரிந்து சாம்பலான 50 ஆயிரம் வாக்கு இயந்திரங்கள்...  
2  முக்கிய இந்திய நாளிதழ்களில், இன்று வெளியான பிர...  
3  2019ஆம் ஆண்டு இறுதியில் கோவிட் 19 என்று அழைக்க...  
4  திமுக ஆட்சிக்கு வரப்போவதில்லை என்பதால் பலவிதமா...  
Data loaded successfully. 

In [ ]:
def summarize_article(article):
    try:
        # Tokenize input article and ensure it's on the correct device (GPU or CPU)
        inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
        print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

        # Check and adjust for max length
        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]


        inp = inp.to(device)

        # Check tensor shape
        print(f"Input tensor shape: {inp.shape}")

        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]

        model.eval()
        model.to(device)
        model_output= model.generate(inp, use_cache=True, num_beams=4, max_length=60, min_length=20, length_penalty=1.0, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2ta>"))

        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"

df5['Summary'] = df5['Article'].apply(summarize_article)

print(df5[['Article', 'Summary']].head())
df5.to_csv("summarized_articles_Tamil.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 524]), on device: cpu
Input tensor shape: torch.Size([1, 524])
Input tensor shape: torch.Size([1, 482]), on device: cpu
Input tensor shape: torch.Size([1, 482])
Input tensor shape: torch.Size([1, 1009]), on device: cpu
Input tensor shape: torch.Size([1, 1009])
Input tensor shape: torch.Size([1, 801]), on device: cpu
Input tensor shape: torch.Size([1, 801])
Input tensor shape: torch.Size([1, 1245]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 238]), on device: cpu
Input tensor shape: torch.Size([1, 238])
Input tensor shape: torch.Size([1, 524]), on device: cpu
Input tensor shape: torch.Size([1, 524])
Input tensor shape: torch.Size([1, 943]), on device: cpu
Input tensor shape: torch.Size([1, 943])
Input tensor shape: torch.Size([1, 944]), on device: cpu
Input tensor shape: torch.Size([1, 944])
Input tensor shape: torch.Size([1, 680]), on device: cpu
Input tensor shape: torch.Size([1, 680])
Input tensor sha

In [ ]:
file_path_6= 'urls_and_articles_summary_telugu.csv'
try:
    df6 = pd.read_csv(file_path_6, encoding='utf-8-sig')
    print(df6.head())
except FileNotFoundError:
    print(f"The file {file_path_6} was not found.")
except UnicodeDecodeError:
    print("Unicode decode error: Try changing encoding to 'utf-8'.")

# Additional check for the contents of 'df' for debugging
print("Data loaded successfully. Columns:", df6.columns)
print("First few rows of data:", df6.head())

                                          URL Source  \
0          https://www.bbc.com/telugu/india-53834486   
1  https://www.bbc.com/telugu/international-52816420   
2          https://www.bbc.com/telugu/india-42666097   
3          https://www.bbc.com/telugu/india-53153796   
4          https://www.bbc.com/telugu/india-50102599   

                                             Article  \
0  ప్రధాని నరేంద్ర మోదీ ప్రైమ్ మినిస్టర్స్ సిటిజన...   
1  2020 అధ్యక్ష ఎన్నికల్లో జోక్యం చేసుకోవడానికి ట...   
2  సుప్రీంకోర్టు నలుగురు సుప్రీంకోర్టు సీనియర్ న్...   
3  పాకిస్తాన్, చైనా, నేపాల్‌లతో భారత సరిహద్దు వివ...   
4  హత్యకు గురయిన విలేకరి కాతా సత్యనారాయణ నెల్లూరు...   

                                             Summary  
0  పీఎం కేర్స్ ఫండ్‌ గురించి విపక్షాలు, యాక్టివిస...  
1  మొట్టమొదటిసారిగా అమెరికా అధ్యక్షుడు డోనల్డ్‌ ట...  
2  దేశ న్యాయవ్యవస్థ దిద్దుబాటు చర్యలు చేపట్టాల్సి...  
3  లిపులేఖ్, కాలాపానీల విషయమై భారత్, నేపాల్‌‌లకు ...  
4  ఆంధ్రప్రదేశ్‌లో కొన్ని నెలలుగా జర్నలిస్టులపై ద..

In [ ]:
def summarize_article(article):
    try:
        # Tokenize input article and ensure it's on the correct device (GPU or CPU)
        inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
        print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

        # Check and adjust for max length
        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]


        inp = inp.to(device)

        # Check tensor shape
        print(f"Input tensor shape: {inp.shape}")

        max_input_length = model.config.max_position_embeddings
        if inp.shape[1] > max_input_length:
            inp = inp[:, :max_input_length]

        model.eval()
        model.to(device)
        model_output= model.generate(inp, use_cache=True, num_beams=4, max_length=60, min_length=20, length_penalty=1.0, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2te>"))

        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"

df6['Summary'] = df6['Article'].apply(summarize_article)

print(df6[['Article', 'Summary']].head())
df6.to_csv("summarized_articles_Telugu.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 1315]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 1324]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 1284]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 584]), on device: cpu
Input tensor shape: torch.Size([1, 584])
Input tensor shape: torch.Size([1, 1001]), on device: cpu
Input tensor shape: torch.Size([1, 1001])
Input tensor shape: torch.Size([1, 1320]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 482]), on device: cpu
Input tensor shape: torch.Size([1, 482])
Input tensor shape: torch.Size([1, 840]), on device: cpu
Input tensor shape: torch.Size([1, 840])
Input tensor shape: torch.Size([1, 1164]), on device: cpu
Input tensor shape: torch.Size([1, 1024])
Input tensor shape: torch.Size([1, 560]), on device: cpu
Input tensor shape: torch.Size([1, 560])
Input te